In [1]:
# IMPORTS
import numpy as np
import numpy.linalg
import numpy.matlib
from numpy.linalg import norm
from numpy.linalg import lstsq

from scipy import interpolate
from scipy import signal
from sklearn.preprocessing import PolynomialFeatures

import csv   
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
# LOAD DATA

# 1) LOAD DYNAMICS AND TIME ARRAYS

filepath = 'data_files/Lorenz63_dynamics.csv'
dynamics = np.genfromtxt(filepath, delimiter=',')

t = dynamics[0]
data = dynamics[1:]
n_states = int(data.shape[0] / 2)
X = data[0:n_states]
DX = data[n_states:]
t_samples = len(t)
dt = np.mean(np.diff(t))

noise_lvls = [0.001,0.01,0.1,1]
n_noise_lvls = len(noise_lvls) 

failed_samples_state = []
failed_samples = []

In [3]:
## Define functions
def kernel_weights(u,kernel_type = 'None'):
    
    if kernel_type == 'Epanechnikov':
        w = 3 / 4 *(1 - u **2)
    else:
        w = np.ones(len(u))

    return w

def locpoly_reg(y, t, t0, degree = 2, h = 1, kernel = 'None'):
    
    t_t0 = t - t0
    nn_indx = np.argwhere(abs(t_t0) < h)
    nn = len(nn_indx)
    y_nn = y[nn_indx]
    t_nn = t_t0[nn_indx]
    t_lpr = t[nn_indx]

    #Basis matrix of polynomial degree d
    poly = PolynomialFeatures(degree)
    B = poly.fit_transform(t_nn)

    #Compute the weights according to specific kernel function
    u = t_nn / h
    w = kernel_weights(u.flatten(), kernel)
    W = np.diag(w)

    #Solve local polynomail regression
    WB = np.dot(W,B)
    Wy = np.dot(W,y_nn)
    theta = lstsq(WB, Wy, rcond=-1)[0]
    y_lpr = np.dot(B,theta)
    
    #Compute the residual sum  of squares (SSE)
    SSE = norm(y_nn - y_lpr)**2

    #Compute the smoother matrix and degrees of freedom
    BtWB = np.dot(B.T,WB)
    K = lstsq(BtWB,B.T, rcond=-1)[0]
    Sh = np.dot(B,K)
    df = np.trace(Sh)
    
    #Compute GCV function
    GCVh = nn * SSE / (nn - df)**2
    
    return [(theta[0], theta[1]), (t_lpr, y_lpr), (SSE, df, nn), GCVh]

In [4]:
## Define parameters and allocate variables

#Range of bandwidths
hmin = 3 * dt
hmax = 0.2 * t[-1]
dh = dt
h_range = np.arange(hmin,hmax,dh)
n_h = len(h_range)

#Degree of the local polynomial
poly_d = 2

#Kernel type
kernel_type = 'Epanechnikov'

#Variable allocation
x_hat = np.zeros((t_samples,n_states))
dx_hat = np.zeros((t_samples,n_states))

mean_X_error = np.zeros((n_states, n_noise_lvls))
mean_dX_error = np.zeros((n_states, n_noise_lvls))

var_X_error = np.zeros((n_states, n_noise_lvls))
var_dX_error = np.zeros((n_states, n_noise_lvls))  

#Save results
write_data = True

In [5]:
for noise, sigma in enumerate(noise_lvls):

    # 2) LOAD OBSERVATION SAMPLES
    Y_samples = []
    noise_lvl = str(sigma)

    for state in range(n_states):

        filepath = 'data_files/Lorenz63_samples_noise_' + noise_lvl +'_Y'+ str(state+1) + '.csv'
        y = np.genfromtxt(filepath, delimiter=',')

        Y_samples.append(y)

    n_realizations = Y_samples[0].shape[0]

    ################### DENOISING AND NUMERICAL DIFFERENTIATION - SAVITZKY-GOLAY FILTER ###################
    trim = int(np.floor(5 / 100 * t_samples))
    error_X = np.zeros((n_realizations,n_states))
    error_dX = np.zeros((n_realizations,n_states))
    tt_samples = t_samples - 2 * trim
    Yhat_sgf = np.zeros((n_realizations, tt_samples))
    DYhat_sgf = np.zeros((n_realizations, tt_samples))

    ## Loop over each state
    for state in range(n_states):

        ##Loop over each realization
        for realization in range(n_realizations):

            y = Y_samples[state][realization,:]

            ## Loop over each time sample
            for sample in range(t_samples):

                #Data sample i
                y0 = y[sample]
                t0 = t[sample]

                GCV = []

                #Loop over different bandwidths
                for i,h in enumerate(h_range):

                    #Locally weighted polynomial regression
                    lpr = locpoly_reg(y, t, t0, poly_d, h, kernel = kernel_type)
                    GCVh = lpr[3]
                    GCV.append(GCVh)

                #Select the optimal bandwidth and run local polynomial regression again
                min_GCV_idx = np.argmin(GCV)
                h_opt = h_range[min_GCV_idx]

                lpr_opt = locpoly_reg(y, t, t0, poly_d, h_opt, kernel = kernel_type)
                #t_lpr, y_lpr = lpr_opt[1]
                x_hat[sample,state], dx_hat[sample,state] = lpr_opt[0]

            Yhat_sgf[realization,:] = x_hat[trim:-trim,state]  
            DYhat_sgf[realization,:] = dx_hat[trim:-trim,state]  

            error_X[realization,state] = norm(X[state,trim:-trim] - Yhat_sgf[realization,:]) / norm(X[state,trim:-trim])
            error_dX[realization,state] = norm(DX[state,trim:-trim] - DYhat_sgf[realization,:]) / norm(DX[state,trim:-trim])
            print('State: ' + str(state+1) + ' | ' + 'Realization: ' + str(realization))

        if write_data:
            filename_y = 'data_files/LOWESS/Lorenz63_LOWESS_' + noise_lvl + '_Y' + str(state+1) + '.csv'
            filename_dy = 'data_files/LOWESS/Lorenz63_LOWESS_' + noise_lvl + '_dY' + str(state+1) + '.csv'
            with open(filename_y, 'w') as csvfile:   
                # creating a csv writer object   
                csvwriter = csv.writer(csvfile)   
                # writing the data rows   
                csvwriter.writerows(Yhat_sgf)
            with open(filename_dy, 'w') as csvfile:   
                # creating a csv writer object   
                csvwriter = csv.writer(csvfile)   
                # writing the data rows   
                csvwriter.writerows(DYhat_sgf)

        mean_X_error[state, noise] = error_X[:,state].mean()
        mean_dX_error[state, noise] = error_dX[:,state].mean()
        var_X_error[state, noise] = error_X[:,state].var()
        var_dX_error[state, noise] = error_dX[:,state].var()

State: 1 | Realization: 0
State: 1 | Realization: 1
State: 1 | Realization: 2
State: 1 | Realization: 3
State: 1 | Realization: 4
State: 1 | Realization: 5
State: 1 | Realization: 6
State: 1 | Realization: 7
State: 1 | Realization: 8
State: 1 | Realization: 9
State: 1 | Realization: 10
State: 1 | Realization: 11
State: 1 | Realization: 12
State: 1 | Realization: 13
State: 1 | Realization: 14
State: 1 | Realization: 15
State: 1 | Realization: 16
State: 1 | Realization: 17
State: 1 | Realization: 18
State: 1 | Realization: 19
State: 1 | Realization: 20
State: 1 | Realization: 21
State: 1 | Realization: 22
State: 1 | Realization: 23
State: 1 | Realization: 24
State: 1 | Realization: 25
State: 1 | Realization: 26
State: 1 | Realization: 27
State: 1 | Realization: 28
State: 1 | Realization: 29
State: 1 | Realization: 30
State: 1 | Realization: 31
State: 1 | Realization: 32
State: 1 | Realization: 33
State: 1 | Realization: 34
State: 1 | Realization: 35
State: 1 | Realization: 36
State: 1 | 

State: 1 | Realization: 5
State: 1 | Realization: 6
State: 1 | Realization: 7
State: 1 | Realization: 8
State: 1 | Realization: 9
State: 1 | Realization: 10
State: 1 | Realization: 11
State: 1 | Realization: 12
State: 1 | Realization: 13
State: 1 | Realization: 14
State: 1 | Realization: 15
State: 1 | Realization: 16
State: 1 | Realization: 17
State: 1 | Realization: 18
State: 1 | Realization: 19
State: 1 | Realization: 20
State: 1 | Realization: 21
State: 1 | Realization: 22
State: 1 | Realization: 23
State: 1 | Realization: 24
State: 1 | Realization: 25
State: 1 | Realization: 26
State: 1 | Realization: 27
State: 1 | Realization: 28
State: 1 | Realization: 29
State: 1 | Realization: 30
State: 1 | Realization: 31
State: 1 | Realization: 32
State: 1 | Realization: 33
State: 1 | Realization: 34
State: 1 | Realization: 35
State: 1 | Realization: 36
State: 1 | Realization: 37
State: 1 | Realization: 38
State: 1 | Realization: 39
State: 1 | Realization: 40
State: 1 | Realization: 41
State:

State: 1 | Realization: 10
State: 1 | Realization: 11
State: 1 | Realization: 12
State: 1 | Realization: 13
State: 1 | Realization: 14
State: 1 | Realization: 15
State: 1 | Realization: 16
State: 1 | Realization: 17
State: 1 | Realization: 18
State: 1 | Realization: 19
State: 1 | Realization: 20
State: 1 | Realization: 21
State: 1 | Realization: 22
State: 1 | Realization: 23
State: 1 | Realization: 24
State: 1 | Realization: 25
State: 1 | Realization: 26
State: 1 | Realization: 27
State: 1 | Realization: 28
State: 1 | Realization: 29
State: 1 | Realization: 30
State: 1 | Realization: 31
State: 1 | Realization: 32
State: 1 | Realization: 33
State: 1 | Realization: 34
State: 1 | Realization: 35
State: 1 | Realization: 36
State: 1 | Realization: 37
State: 1 | Realization: 38
State: 1 | Realization: 39
State: 1 | Realization: 40
State: 1 | Realization: 41
State: 1 | Realization: 42
State: 1 | Realization: 43
State: 1 | Realization: 44
State: 1 | Realization: 45
State: 1 | Realization: 46
S

State: 1 | Realization: 15
State: 1 | Realization: 16
State: 1 | Realization: 17
State: 1 | Realization: 18
State: 1 | Realization: 19
State: 1 | Realization: 20
State: 1 | Realization: 21
State: 1 | Realization: 22
State: 1 | Realization: 23
State: 1 | Realization: 24
State: 1 | Realization: 25
State: 1 | Realization: 26
State: 1 | Realization: 27
State: 1 | Realization: 28
State: 1 | Realization: 29
State: 1 | Realization: 30
State: 1 | Realization: 31
State: 1 | Realization: 32
State: 1 | Realization: 33
State: 1 | Realization: 34
State: 1 | Realization: 35
State: 1 | Realization: 36
State: 1 | Realization: 37
State: 1 | Realization: 38
State: 1 | Realization: 39
State: 1 | Realization: 40
State: 1 | Realization: 41
State: 1 | Realization: 42
State: 1 | Realization: 43
State: 1 | Realization: 44
State: 1 | Realization: 45
State: 1 | Realization: 46
State: 1 | Realization: 47
State: 1 | Realization: 48
State: 1 | Realization: 49
State: 1 | Realization: 50
State: 1 | Realization: 51
S

In [6]:
#ERROR ANALYSIS

temp = [['$\sigma$ = ' + str(x)] * n_states for x in noise_lvls]
noise_indx = np.array([item for sublist in temp for item in sublist])

state_indx = np.array(['state x' + str(x+1) for x in range(n_states)] * n_noise_lvls)

arrays = [noise_indx, state_indx]
data = np.vstack((mean_X_error.T.flatten(), var_X_error.T.flatten(), mean_dX_error.T.flatten(), var_dX_error.T.flatten())).T

col_names = ['mean error x', 'var. error x', 'mean error dx', 'var. error dx']
df = pd.DataFrame(data, index=arrays, columns=col_names)
df

mean error x  var. error x  mean error dx  \
$\sigma$ = 0.001 state x1      0.000090  3.234458e-11       0.009150   
                 state x2      0.000084  3.582132e-11       0.014895   
                 state x3      0.000034  5.543322e-12       0.011941   
$\sigma$ = 0.01  state x1      0.000865  2.508243e-09       0.012370   
                 state x2      0.000701  2.509128e-09       0.015742   
                 state x3      0.000284  2.968940e-10       0.012499   
$\sigma$ = 0.1   state x1      0.007821  4.036978e-07       0.070187   
                 state x2      0.006473  2.615160e-07       0.048036   
                 state x3      0.002630  3.702319e-08       0.036356   
$\sigma$ = 1     state x1      0.069179  3.610600e-05       0.584451   
                 state x2      0.056863  2.634138e-05       0.384722   
                 state x3      0.023798  4.436510e-06       0.290772   

                           var. error dx  
$\sigma$ = 0.001 state x1   1.472648e-09  
                 state x2   9.857603e-10  
                 state x3   4.094848e-10  
$\sigma$ = 0.01  state x1   1.814260e-07  
                 state x2   1.214485e-07  
                 state x3   4.101093e-08  
$\sigma$ = 0.1   state x1   3.534315e-05  
                 state x2   1.302062e-05  
                 state x3   7.340619e-06  
$\sigma$ = 1     state x1   3.320405e-03  
                 state x2   1.413557e-03  
                 state x3   8.400460e-04